<a href="https://colab.research.google.com/github/onyxws/ONYX-SYNC/blob/main/labs/02-BigQuery/notebooks/BigQuery_exercises.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Google BigQuery

## Walkthrough

### Authentication

In [1]:
import os

PROJECT_NAME = "labcloud-451519" # REPLACE WITH YOUR PROJECT ID


#### For Colab users (recommended)

In [2]:
from google.colab import auth
auth.authenticate_user()
print("Authenticated")

Authenticated


#### For Jupyter users ONLY (more challenging)

Follow [this](https://cloud.google.com/iam/docs/creating-managing-service-account-keys) guide to create the JSON with the service account key. Then, make sure to replace "PATH_TO_CREDENTIALS_FILE" with the *absolute* path to the JSON service account key (e.g., "C:/Users/John/credentials.json").

In [ ]:
# !pip install google-cloud-bigquery
# !pip install db-dtypes

In [ ]:
# os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = PATH_TO_KEY

### Connect to BigQuery

In [3]:
import pandas as pd
from google.cloud import bigquery

# Create a "Client" object
client = bigquery.Client(project=PROJECT_NAME)

To run the cells below, you must first create a new dataset in BigQuery and upload the data from week 2 (`Teams.csv` and `Players.csv`) as tables. Since the CSV files are well-formed, you can automatically infer the schema when loading the data.

In [4]:
# Create a reference to the World Cup dataset
dataset_ref = client.dataset("world_cup", project=PROJECT_NAME)  # CHANGE WITH YOUR OWN DATASET (NOT TABLE) NAME

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

In [5]:
# List the tables in the dataset
tables = list(client.list_tables(dataset))
for table in tables:
    print(table.table_id)

players
players_and_teams
teams


In [6]:
# Create a reference to the "Teams" table and fetch the table
table_ref = dataset_ref.table("teams") # Make sure it matches the table name printed above (case sensitive)
teams_table = client.get_table(table_ref)

# Same for the "Players" table
table_ref = dataset_ref.table("players") # Make sure it matches the table name printed above (case sensitive)
players_table = client.get_table(table_ref)

In [7]:
# Display the schema of the "Teams" table
players_table.schema

[SchemaField('surname', 'STRING', 'NULLABLE', None, None, (), None),
 SchemaField('team', 'STRING', 'NULLABLE', None, None, (), None),
 SchemaField('position', 'STRING', 'NULLABLE', None, None, (), None),
 SchemaField('minutes', 'INTEGER', 'NULLABLE', None, None, (), None),
 SchemaField('shots', 'INTEGER', 'NULLABLE', None, None, (), None),
 SchemaField('passes', 'INTEGER', 'NULLABLE', None, None, (), None),
 SchemaField('tackles', 'INTEGER', 'NULLABLE', None, None, (), None),
 SchemaField('saves', 'INTEGER', 'NULLABLE', None, None, (), None)]

In [8]:
# Show a preview of the "Teams" table using pandas
client.list_rows(teams_table, max_results=5).to_dataframe()

,team,ranking,games,wins,draws,losses,goalsFor,goalsAgainst,yellowCards,redCards
0,Honduras,38,3,0,1,2,0,3,7,0
1,North Korea,105,3,0,0,3,1,12,2,0
2,Greece,13,3,1,0,2,2,5,5,0
3,Cameroon,19,3,0,0,3,2,5,5,0
4,New Zealand,78,3,0,3,0,2,2,6,0


### Example of queries

In [9]:
q1 = """
select *
from %s.world_cup.teams
where ranking < 20
order by ranking
""" %PROJECT_NAME

query_job = client.query(q1)
query_job.to_dataframe()

,team,ranking,games,wins,draws,losses,goalsFor,goalsAgainst,yellowCards,redCards
0,Brazil,1,5,3,1,1,9,4,7,2
1,Spain,2,6,5,0,1,7,2,3,0
2,Portugal,3,4,1,2,1,7,1,8,1
3,Netherlands,4,6,6,0,0,12,5,15,0
4,Italy,5,3,0,2,1,4,5,5,0
5,Germany,6,6,4,0,2,13,3,8,1
6,Argentina,7,5,4,0,1,10,6,7,0
7,England,8,4,1,2,1,3,5,6,0
8,France,9,3,0,1,2,1,4,6,1
9,Greece,13,3,1,0,2,2,5,5,0


In [10]:
# Join the two tables
q2 = """
select teams.*, players.surname, players.position, players.shots,
players.minutes, players.passes, players.tackles, players.saves
from %s.world_cup.teams teams
join %s.world_cup.players players
on teams.team = players.team
""" %(PROJECT_NAME, PROJECT_NAME)

query_job = client.query(q2)
query_job.to_dataframe().head(10)

,team,ranking,games,wins,draws,losses,goalsFor,goalsAgainst,yellowCards,redCards,surname,position,shots,minutes,passes,tackles,saves
0,Honduras,38,3,0,1,2,0,3,7,0,Chavez,defender,0,270,62,3,0
1,Honduras,38,3,0,1,2,0,3,7,0,Izaguirre,defender,0,180,51,8,0
2,Honduras,38,3,0,1,2,0,3,7,0,Mauricio Sabilluen,defender,0,90,38,5,0
3,Honduras,38,3,0,1,2,0,3,7,0,Mendoza,defender,0,180,51,3,0
4,Honduras,38,3,0,1,2,0,3,7,0,Jerry Palacios,forward,0,84,19,0,0
5,Honduras,38,3,0,1,2,0,3,7,0,Walter Martinez,forward,0,125,43,1,0
6,Honduras,38,3,0,1,2,0,3,7,0,Valladares,goalkeeper,0,270,51,0,12
7,Honduras,38,3,0,1,2,0,3,7,0,Espinoza,midfielder,0,135,36,4,0
8,Honduras,38,3,0,1,2,0,3,7,0,Guevara,midfielder,0,156,60,6,0
9,Honduras,38,3,0,1,2,0,3,7,0,Thomas,midfielder,0,114,37,5,0


### Basic Queries

In [11]:
query = "SELECT * FROM {}.world_cup.players LIMIT 5".format(PROJECT_NAME)
query_job = client.query(query)
query_job.to_dataframe().head(10)

,surname,team,position,minutes,shots,passes,tackles,saves
0,Otamendi,Argentina,defender,250,0,141,6,0
1,Beauchamp,Australia,defender,90,0,33,2,0
2,Carney,Australia,defender,180,0,56,4,0
3,Gilberto,Brazil,defender,33,0,6,4,0
4,Assou-Ekotto,Cameroon,defender,270,0,116,5,0


In [12]:
query = "SELECT * FROM {}.world_cup.teams LIMIT 5".format(PROJECT_NAME)
query_job = client.query(query)
query_job.to_dataframe().head(10)

,team,ranking,games,wins,draws,losses,goalsFor,goalsAgainst,yellowCards,redCards
0,Honduras,38,3,0,1,2,0,3,7,0
1,North Korea,105,3,0,0,3,1,12,2,0
2,Greece,13,3,1,0,2,2,5,5,0
3,Cameroon,19,3,0,0,3,2,5,5,0
4,New Zealand,78,3,0,3,0,2,2,6,0


In [13]:
query = """SELECT surname, team, minutes, passes
            FROM {}.world_cup.players
            WHERE team="Switzerland" AND surname LIKE "%er%"
            ORDER BY surname""".format(PROJECT_NAME)
query_job = client.query(query)
query_job.to_dataframe().head(10)

,surname,team,minutes,passes
0,Derdiyok,Switzerland,191,47
1,Fernandes,Switzerland,212,56
2,Inler,Switzerland,270,138
3,Lichtsteiner,Switzerland,270,80
4,Senderos,Switzerland,36,9
5,Ziegler,Switzerland,270,71
6,von Bergen,Switzerland,234,79


In [14]:
query = """SELECT team, COUNT(*) AS number_players_listed
            FROM {}.world_cup.players
            GROUP BY team
            HAVING number_players_listed<20
            ORDER BY number_players_listed DESC
            LIMIT 5""".format(PROJECT_NAME)
query_job = client.query(query)
query_job.to_dataframe().head(10)

,team,number_players_listed
0,England,19
1,Australia,19
2,Brazil,19
3,France,19
4,Denmark,19


In [15]:
query = """SELECT surname, position, minutes, T.team, ranking, games, wins, draws, losses
            FROM {}.world_cup.players as P
            JOIN {}.world_cup.teams AS T
            ON P.team=T.team
            WHERE surname <"C"
            LIMIT 5""".format(PROJECT_NAME, PROJECT_NAME)
query_job = client.query(query)
query_job.to_dataframe().head(10)

,surname,position,minutes,team,ranking,games,wins,draws,losses
0,Beauchamp,defender,90,Australia,20,3,1,1,1
1,Assou-Ekotto,defender,270,Cameroon,19,3,0,0,3
2,Bassong,defender,163,Cameroon,19,3,0,0,3
3,Bong,defender,56,Cameroon,19,3,0,0,3
4,Ashley Cole,defender,360,England,8,4,1,2,1


## Exercise 1 - World Cup datasets

### Simple Queries
The first three questions are already solved for you, so that you have concrete examples of queries. Try to solve the remaining ones!

*1)  Which player on a team ending with "ia" played less than 200 minutes and made more than 100 passes? Return the player's surname and team.*

**Hint**: To check if attribute A contains a (sub)string S, use the LIKE keyword (e.g. `A like '%S%'`). The % sign indicates a wildcard.

In [ ]:
# Your query goes here
query = """YOUR_QUERY_HERE""".format(PROJECT_NAME)
query_job = client.query(query)
query_job.to_dataframe().head(10)

*2) Find all players who made more than 20 shots. Return all player information in descending order of shots made.*

**Hint**: Sorting results is done via the ORDER BY keyword. The default order is ascending (ASC). If you want descending order, use DESC (e.g. `ORDER BY column_1, column_2 DESC`).

In [ ]:
# Your query goes here
query = """YOUR_QUERY_HERE""".format(PROJECT_NAME)
query_job = client.query(query)
query_job.to_dataframe().head(10)

*3) Which team has the highest average number of passes per minute played? Return the team's name and average number of passes per minute.*

**Hint #1**: You can compute a team's average number of passes per minute played by dividing the total number of passes by the total number of minutes. To force floating point division, multiply one operand by 1.0.

**Hint #2**: Consider using the LIMIT keyword.

In [ ]:
# Your query goes here
query = """YOUR_QUERY_HERE""".format(PROJECT_NAME)
query_job = client.query(query)
query_job.to_dataframe().head(10)

*4) Find the goalkeepers of teams that played more than four games. List the surname of the goalkeeper, the team, and the number of minutes the goalkeeper played.*

**Hint**: Use the `PlayersExt` table.

In [ ]:
# Your query goes here
query = """YOUR_QUERY_HERE""".format(PROJECT_NAME)
query_job = client.query(query)
query_job.to_dataframe().head(10)

*5) How many players on a team with a ranking lower than 10 played more than 350 minutes? Return a single number in a column named "superstar".*

**Hint**: To rename a column, use the AS keyword (e.g. `SELECT column_1 AS label`).

In [ ]:
# Your query goes here
query = """YOUR_QUERY_HERE""".format(PROJECT_NAME)
query_job = client.query(query)
query_job.to_dataframe().head(10)

*6) What is the average number of passes made by forwards? What about midfielders? Write one query that returns both values with the corresponding position.*

**Hint**: Use the GROUP BY keyword. GROUP BY statements are often used in conjuction with aggregate functions like AVG(), SUM() or COUNT().

In [ ]:
# Your query goes here
query = """YOUR_QUERY_HERE""".format(PROJECT_NAME)
query_job = client.query(query)
query_job.to_dataframe().head(10)

### Advanced Queries
Now, on to more challenging questions...

*1) Find all pairs of teams that have the same number of `goalsFor` as well as the same number of `goalsAgainst` as each other. Return the team pairs and their respective numbers of `goalsFor` and `goalsAgainst` (make sure to return each pair only once!).*

**Hint**: You basically need to do a "self join" of the `Teams` table. For that, you need to join different name aliases of the same table. Check [this page](https://www.w3schools.com/sql/sql_join_self.asp) for help.

In [ ]:
# Your query goes here
query = """YOUR_QUERY_HERE""".format(PROJECT_NAME, PROJECT_NAME)
query_job = client.query(query)
query_job.to_dataframe().head(10)

*2) Find all teams with a ranking below 30 where no player has made more than 150 passes. Return the team's name and ranking.*

**Hint #1**: Consider using the HAVING keyword.

**Hint #2**: You may also want to look up nested queries.

In [ ]:
# Your query goes here
query = """YOUR_QUERY_HERE""".format(PROJECT_NAME)
query_job = client.query(query)
query_job.to_dataframe().head(10)

*3) Which team has the highest ratio of goalsFor to goalsAgainst?*

In [ ]:
# Your query goes here
query = """YOUR_QUERY_HERE""".format(PROJECT_NAME)
query_job = client.query(query)
query_job.to_dataframe().head(10)

*4) Find all teams whose defenders averaged more than 150 passes. Return the team and average number of passes by defenders, in descending order of average passes.*

In [ ]:
# Your query goes here
query = """YOUR_QUERY_HERE""".format(PROJECT_NAME)
query_job = client.query(query)
query_job.to_dataframe().head(10)

## Exercise 2 - Google Cloud datasets

Let's build some queries for the `accidents_2016` data from the NHTSA traffic facilities dataset, which is publicly available on Google BigQuery.

### Setting up

In [ ]:
# Create a client for public datasets
client = bigquery.Client(project="bigquery-public-data")

# Create a reference to the traffic facilities dataset
traffic_ref = client.dataset("nhtsa_traffic_fatalities", project="bigquery-public-data")

# API request - fetch the dataset
traffic_dataset = client.get_dataset(traffic_ref)

In [ ]:
# Construct a reference to the "accident_2016" table
accidents_ref = traffic_ref.table("accident_2016")

# API request - fetch the table
accidents_table = client.get_table(accidents_ref)

In [ ]:
# Display schema
accidents_table.schema

In [ ]:
# Show a preview of the "accident_2016" table
client.list_rows(accidents_table, max_results=5).to_dataframe()

### Questions

You can access the table using `from bigquery-public-data.nhtsa_traffic_fatalities.accident_2016`.

1) How many crashes happened in each month of 2016?

In [ ]:
q3 = """YOUR_QUERY_HERE"""

query_job_3 = client.query(q3)
query_job_3.to_dataframe()

2) Let's call the accidents involving more than 5 motor vehichles in transport involved as "massive_crash", and all others as "small crash". Return the number of each type of crash for each weather condition.

**Hint**: Use columns `atmospheric_conditions_1_name` and `number_of_motor_vehicles_in_transport_mvit`.

In [ ]:
q4 = """YOUR_QUERY_HERE"""

query_job_4 = client.query(q4)
query_job_4.to_dataframe()

3) Find the top 5 states with the highest number of drunk drivers involved in accidents.

In [ ]:
q5 = """YOUR_QUERY_HERE"""

query_job_5 = client.query(q5)
query_job_5.to_dataframe()

4) Let's find out the states where using a cell phone is a major cause of accidents. Return the first 5 state names along with the number of crashes where the driver was distracted "while manipulating cellular phone".

**Hint**: You need to join the `distract_2016` and `accident_2016` tables.

In [ ]:
q6 = """YOUR_QUERY_HERE"""

query_job_6 = client.query(q6)
query_job_6.to_dataframe()

5) Sometimes, a "live animal" is a reason that a driver maneuvers and causes an accident. In which months of the year do such accidents happen the most? Return the month and the number of accidents.

**Hint**: You need to join the `maneuver_2016` and `accident_2016` tables.

In [ ]:
q7 = """YOUR_QUERY_HERE"""

query_job_7 = client.query(q7)
query_job_7.to_dataframe()

6) In the state of Alaska, in which months do "poor road conditions" cause the most accidents? Return the month and the number of accidents.

**Hint**: Again, you will need the `maneuver_2016` table.

In [ ]:
q8 = """YOUR_QUERY_HERE"""

query_job_8 = client.query(q8)
query_job_8.to_dataframe()

7) Sometimes, a factor of the accident is that the vehicle has a problem with the "brake system". Filter the accidents with this factor and return the `driver_maneuvered_to_avoid_name` column along with the related number of accidents.

**Hint**: You need to join the `maneuver_2016` and `factor_2016` tables.

In [ ]:
q9 = """YOUR_QUERY_HERE"""

query_job_9 = client.query(q9)
query_job_9.to_dataframe()

8) Sometimes, a non-motorist is involved in a crash. For each state, find the number of crashes in which an "inattentive" person was involved. Return the state name and number of crashes.

**Hint**: You need to join the `nmcrash_2016` and `accident_2016` tables.

In [ ]:
q10 = """YOUR_QUERY_HERE"""

query_job_10 = client.query(q10)
query_job_10.to_dataframe()